# Creating the main_cell & JJ and JJ-Array

## Positive markers and frame by Qiskit-Metal

In [4]:
import gdspy
from importlib import reload
from itertools import product
reload(gdspy)
lib = gdspy.GdsLibrary()


#######################################################################################

#create base layer with main ground and markers
main_cell = lib.new_cell('main_cell')

unground = gdspy.Rectangle((-4500,-4500), (4500, 4500), layer=2)
main_cell.add(unground)

      
#logo and the device name
device_name = gdspy.Text("FX23-W3-01-V3.3", 150, (800, -4350), layer=3)
logo = gdspy.Text("Andersen Lab/QuTech", 150, (250, -4500), layer=3)
main_cell.add(logo)
main_cell.add(device_name)

#######################################################################################

##### N SHOULD BE EVEN!!!!!!!!!!
# N is a number of JJ in the array
N = 100

if int(N / 2) % 2 != 0:
    raise ValueError('N / 2 should be even!!!')

delta_x = 0.5
# delta_x_base = 1.5
delta_x_base = 1.5
# nanowire_length is  the length between nanowire's pads
nanowire_length = 130 + 3
real_nanowire_length = nanowire_length - delta_x * 2 - delta_x_base * 2

# very important shift at x direction
SHIFT = - nanowire_length / 2
nanowire_width = 1.3

delta = 0.410  # initial width of pad

nu_cell = 0

start_x = 0
start_y = 0

stop_x = start_x + nanowire_length
stop_y = start_y

# for vertical and horizontallines
step = 1.2
delta_W = (real_nanowire_length - 2 * step) / (N / 2 - 1)
delta_H = 3
print(delta_W)

corr1 = 1.5
corr2 = 1

u = 1 #undercut length

if delta == 0.410:
    cell = lib.new_cell('jj_array' + str(nu_cell))
    
# Bandage for the left    
    ban1 = gdspy.Rectangle((start_x + delta_x_base - 2.5 + SHIFT - 0.2, start_y - 0.7 / 2 - 0.2-3),
                           (start_x + delta_x_base + delta_x + SHIFT + 0.2, start_y + 0.7 / 2 + 0.2-3),
                          layer=56)
    
    cell.add(ban1)
    
    # left line
    
    left_line = gdspy.Rectangle((start_x + delta_x_base + delta_x+SHIFT-1.6-5.5, start_y - delta / 2-3),
                                (start_x + delta_x_base + delta_x + step + delta+corr2+SHIFT-4.0, start_y + delta / 2-3), layer=54_2)
    cell.add(left_line)
    
    undercut_left_line = gdspy.Rectangle((start_x + delta_x_base + delta_x + step + delta+corr2+SHIFT-4.0, start_y - delta / 2-3),
                                         (start_x + delta_x_base + delta_x + step + delta+corr2+0.7+SHIFT-4.0, start_y + delta / 2-3),
                                         layer=55)
    cell.add(undercut_left_line)
    
# Bandage for the right    
    ban2 = gdspy.Rectangle((stop_x + SHIFT -1.8, stop_y - 3.7),
                           (stop_x + SHIFT -1.8+2.6, stop_y - 3.7 + 1.1), 
                          layer=56)
    cell.add(ban2)
    
    
########################### ADDING JJ ARRAY ########################################################### 
    array_JJ = None
    k = 0 #  center
    p = 0 #  down 
    for i in range(int(N / 2)):
        if i<1:
            start_y=-3
            delta_H = 3.8
            start_x = -7
            corr2 = 8
        elif i<2:
            start_y = -3
            delta_H = 3
            start_x = 0
            corr2 = 1
        else:
            start_y=0
            delta_H = 3
            start_x = 0
            corr2 = 1
        if i<N//2-1:
            vertical_pad = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT,
                 start_y - 6),
                (start_x + delta_x_base + delta_x + step + delta_W * i + delta+SHIFT,
                 start_y + 6),
                layer=54_2)
            vertical_circle_1 = gdspy.Round((start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT +delta/2,
                                             start_y-5),
                                            radius=1.6*delta/2, layer=54_2, number_of_points=64)
            vertical_pad = gdspy.boolean(vertical_pad, vertical_circle_1, 'or', layer=54_2)
            vertical_circle_2 = gdspy.Round((start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT +delta/2,
                                             start_y+5),
                                            radius=1.6*delta/2, layer=54_2, number_of_points=64)
            vertical_pad = gdspy.boolean(vertical_pad, vertical_circle_2, 'or', layer=54_2)


            undercut_vertical = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT,
                 start_y + 6),
                (start_x + delta_x_base + delta_x + step + delta_W * i + delta+SHIFT,
                 start_y + 6 + u*2),

                layer=55)

            array_JJ = gdspy.boolean(array_JJ, vertical_pad, 'or')

            cell.add(vertical_pad)
            cell.add(undercut_vertical)
        if i==N//2-2:
            corr2 = 2.5
        if i % 2 != 0 and i<N//2-1: #and i != int(N / 2) - 1:
            
            if k % 2 == 0:
                horizonatal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta / 2+ 3),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y + delta / 2+ 3 ),
                layer=54_2)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta / 2+ 3),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 + u+SHIFT, start_y + delta / 2+ 3),
                layer=55)
                
            else:
                horizonatal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta / 2+1 ),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y + delta / 2+1 ),
                layer=54_2)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta / 2 +1),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 + u+SHIFT, start_y + delta / 2+1),
                layer=55)
                        
            k +=1
        
            array_JJ = gdspy.boolean(array_JJ, horizonatal_center, 'or')

            cell.add(horizonatal_center)

            cell.add(undercut_horizontal_center)

        if i % 2 == 0 and i<N//2-1:
            
            if p % 2 == 0:
                
                horizonatal_down = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta_H - delta),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H),
                layer=54_2)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 +u+SHIFT, start_y - delta_H - delta),
                layer=55)
                
            else:
                horizonatal_down = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta_H - delta + 1.5),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H + 1.5),
                layer=54_2)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H+1.5),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 +u+SHIFT, start_y - delta_H - delta+1.5),
                layer=55)
                
            p += 1

        
            array_JJ = gdspy.boolean(array_JJ, horizonatal_down, 'or')

            cell.add(horizonatal_down)

            cell.add(undercut_horizontal_center)
            
########################### ADDING JJ ###########################################################
            
deltaJJ = 0.120 # initial width of pad
pad_length_left = 2.4
pad_length_right = 1.7

nu_cellJJ = 0

if deltaJJ == 0.120:
            cell = lib.new_cell('junction_' + str(nu_cellJJ))
    
            left_big_points = [(-15., 0), (-10, 0), (-2., 1.5), (-2., 3), (-15, 3)]
            right_big_points = [(2., -1.5), (15, -1.5), (15., 1.5), (10, 1.5), (2., 0)]
    
            left_big = gdspy.Polygon(left_big_points, layer=53)
            right_big = gdspy.Polygon(right_big_points, layer=53)
    
            left_small = gdspy.Rectangle((-2., 1.5), (-0.2, 3), layer=53)
    
            right_small = gdspy.Rectangle((0.2, -1.5), (2., 0), layer=53)
    

        # Pads of junctions
            left_padJJ = gdspy.Rectangle((-1.2, -2.8), (-1.2 + deltaJJ, 2.3), layer=54)
    
            right_padJJ = gdspy.Rectangle((-3.0, -1.0), (1.2, -1.0 + deltaJJ), layer=54)
    
    
        # parts for undercuts
            undercut_left_pad3 = gdspy.Rectangle((-1.2, 2.3), (-1.2 + deltaJJ, + 3), layer=55)
    
            undercut_right_pad4 = gdspy.Rectangle((1.2, -1.0), (1.9, -1.0 + deltaJJ), layer=55)
    
        #bandages
            ban3 = gdspy.Rectangle((-1.8, 1.2), (-0.6, 3), layer=56)
            ban4 = gdspy.Rectangle((-0.25, -1.5), (2., -0.04), layer=56)
            

    

            cell.add(left_big)
            cell.add(right_big)

            cell.add(left_small)
            cell.add(right_small)

            cell.add(left_padJJ)
            cell.add(right_padJJ)
    
            cell.add(undercut_left_pad3)
            cell.add(undercut_right_pad4)
    
            cell.add(ban3)
            cell.add(ban4)
            



2.583673469387755


In [5]:
gdspy.LayoutViewer()

C:\Users\figenyilmaz\AppData\Local\Temp\ipykernel_9632\1312026100.py:1: DeprecationWarning: [GDSPY] Use of the global library is deprecated.  Pass LayoutViewer a GdsLibrary instance.
  gdspy.LayoutViewer()


<gdspy.viewer.LayoutViewer object .!layoutviewer>

In [6]:
lib.write_gds('FX23_JJ120andArray410_NoMarkers_9by9mm.gds')

# Let's try the GDS file that we just created. We can only use Q1 and see. 
# Create geometry of a chip


In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
from collections import OrderedDict

import warnings
warnings.filterwarnings('ignore')

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs
import matplotlib.pyplot as plt

from qiskit_metal.toolbox_metal import math_and_overrides

from qiskit_metal.qlibrary.core import QComponent

from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors

from qiskit_metal.qlibrary.qubits.fluxoniumEPR import FluxoniumPocket

from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond

from itertools import product

In [5]:
# Each time you create a new quantum circuit design,
# you start by instantiating a QDesign class.

# The design class `DesignPlanar` is best for 2D circuit designs.
design = designs.DesignPlanar()
design.chips.main.material = 'silicon'
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '8.733 um'
design._chips['main']['size']['size_x'] = '9mm'
design._chips['main']['size']['size_y'] = '9mm'

# If you disable the next line with "overwrite_enabled", then you will need to
# delete a component [<component>.delete()] before recreating it.
design.overwrite_enabled = True

gui = MetalGUI(design)

In [3]:
from qiskit_metal.qlibrary.qubits.fluxoniumEPR import FluxoniumPocket

gds_cell_name_jj = 'junction_0'
gds_cell_name_inductor = 'jj_array0'

# Q1 details
options = dict(chip='main', nanowire = False, 
            orientation =90, pos_x = '-2.2mm', pos_y = '-2.3mm',
            gds_cell_name=gds_cell_name_jj,
            gds_cell_inductor=gds_cell_name_inductor ,
               inductor_orientation='-1',
               
    flux_bias_line_options=Dict(make_fbl = True,
                               ), 
               
    charge_line_options=Dict(loc_H = +1, make_cl = True,
                            ),
    readout_line_options=Dict(loc_H = -1, make_rol = True, 
                           )
              )
               
q1 = FluxoniumPocket(design,'Q1', options = dict(
             **options))

gui.rebuild()

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

 

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  vertices = concatenate([

  vertices = concatenate([

  vertices = concatenate([



# Creating the GDS file and yes we are the GOD!

In [4]:
fluxonium_gds = design.renderers.gds
fluxonium_gds.options

{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': '2.0',
 'gds_unit': 0.001,
 'ground_plane': 'True',
 'negative_mask': {'main': []},
 'fabricate': 'False',
 'corners': 'circular bend',
 'tolerance': '0.00001',
 'precision': '0.000000001',
 'width_LineString': '10um',
 'path_filename': '../resources/Fake_Junctions.GDS',
 'junction_pad_overlap': '5um',
 'max_points': '199',
 'cheese': {'datatype': '100',
  'shape': '0',
  'cheese_0_x': '25um',
  'cheese_0_y': '25um',
  'cheese_1_radius': '200um',
  'view_in_file': {'main': {1: True}},
  'delta_x': '100um',
  'delta_y': '100um',
  'edge_nocheese': '300um'},
 'no_cheese': {'datatype': '99',
  'buffer': '25um',
  'cap_style': '2',
  'join_style': '2',
  'view_in_file': {'main': {1: True}}},
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2'}

In [5]:
# Choose file (with gradual junctions or note)
# fluxonium_gds.options['path_filename'] = 'fluxonium_junctions.gds'
# fluxonium_gds.options['path_filename'] = 'fluxonium_junctions_gradual.gds'
fluxonium_gds.options['path_filename'] = 'FX_JJandArrayCircles_9by9mm.gds'

In [6]:
fluxonium_gds.options['fabricate'] = True

In [7]:
# fluxonium_gds.export_to_gds('fluxonium_calibration_structure.gds')
fluxonium_gds.export_to_gds('FluxoniumPocket-Q1-410nmArray120umJJCircles-9x9.gds')

08:47PM 15s WARNING [_give_rotation_center_twopads]: In junction table, component=Q1 with name=inductor has width=0.001 smaller than cell dimension=0.017.
  all_polys = list(multi_poly)

  all_polys = list(multi_poly)



1

In [8]:
gui.main_window.close()

True